<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex14noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex14noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 正規分布の確率密度関数
from scipy.stats import norm
from scipy.stats import multivariate_normal

# 身長体重のデータ
dfHW = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hw.csv', header=0)

----
## 確率密度の推定
----
与えられたデータの背後にある確率分布を推定する（データがどんな確率分布から生み出されたかを推定する）問題を考えます．
ここでは特に，量的な特徴量のみから成るデータを対象として，その背後にある確率密度関数を推定する問題を扱います．


---
### 確率密度（確率分布）の推定とは

次のようなデータ（1000人の身長，単位は[cm]）があります．

In [ ]:
# 1000人の身長
dfHW['height']

「これらのデータは，ある確率分布に従って生成されたものだ」と仮定して，それがどんな確率分布であるかを推定することを考えましょう．例えば，上記のデータが正規分布に従うと仮定して，データからその平均と分散を推定する，ということです（下図参照）．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/density01.png">

この問題は，モデル（データの背後にある確率分布の式，
例では正規分布の式）を仮定して，データからモデルのパラメータ（例では $\mu$ と $\sigma^2$）を決定する，という話になっています．これは，統計学の分野で古くから研究されてきた問題で，**密度推定**（確率密度の推定）と呼ばれる問題の一種です．
機械学習の視点からは，教師なし学習の一種とみなすことができます．

データの密度推定ができれば，次のようなことが可能となります．
1. データの性質を記述できる．分析や理解が容易になる．
1. 推定された分布を用いて新しいデータを生成できる．


モデルとなる確率分布としては正規分布の他にも様々なものがあり得ますが，ここでは，最も典型的なものとして正規分布を取り上げて，この問題についてもう少し考えてみます．



#### ［よだんだよん］

ここでは，モデルとして，正規分布のように少数のパラメータで表されるものを考えています．統計学では，このようなモデルを利用する手法は「パラメトリック(parametric)な手法」と呼ばれています．これに対して，モデルとして特定の確率分布を仮定しない手法もあり，「ノンパラメトリック(non-parametric)な手法」と呼ばれています．
ここで説明しているのは，パラメトリックな密度推定手法です．

---
### 1次元正規分布の当てはめ

データが1次元の正規分布から生成されたと仮定できるときに，その正規分布のパラメータ（平均と分散）を求める問題を考えます．これは，データが与えられたときに，そのデータに最もよく当てはまる1次元正規分布（のパラメータ）を求める問題とも言えます．

#### 1次元正規分布当てはめの問題

**［1次元正規分布当てはめの問題］** $N$個の実数値から成るデータ集合
$$
\{ x_n \in {\cal R} | n = 1, 2, \ldots, N\}
$$
が与えられる．これらのデータがすべて同一の1次元正規分布
$$
p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp{\left( - \frac{(x-\mu)^2}{2\sigma^2}\right)}
$$
に従い，かつ互いに独立に生み出されていると仮定する（注）．
このとき，データに最もよく当てはまるパラメータ $\mu, \sigma^2$ を求めなさい．

<span style="font-size: 75%">
※注: 個々の値が，同一の（平均や分散も同じ）分布から生み出されていると仮定できないと，そもそも平均や分散を一つ決めることができないですね．また，個々の値が独立でない，例えば，1つ目の値に応じて2つ目の値が決まる，というようなことがあってもやはり困ります（後述の最尤推定の話も参照）．このような「独立同一分布」の仮定は統計の問題でよく出てきます．Indenepdent and Identically ditributed の略として，i.i.d. と略記されていることもあります．
</span>

ある考え方（後述します）にしたがって「データに最もよく当てはまる」パラメータ $\mu, \sigma^2$ を求めると，次のようになります．
$$
\begin{aligned}
\mu &= \frac{1}{N}\sum_{n=1}^{N}x_n & (1)\\
\sigma^2 &= \frac{1}{N}\sum_{n=1}^{N}(x_n - \mu)^2 & (2)
\end{aligned}
$$
これらは，それぞれデータの平均（標本平均）と分散（標本分散）ですね．

この結論は，一見当たり前のことのようにも思えます．ですが，本当にこれらが「データに最もよく当てはまる」ものであることはまだ説明できていません．以下，その説明をします．

#### 最尤推定

ある確率分布 $p(x)$ があったときに，あるデータ $x$ がこの確率分布から生成されたと考えることの「尤もらしさ（もっともらしさ）」を表す **尤度**（ゆう度，likelihood）を $\ell(x)$ と表し，次式で定めます．
$$
\ell(x) = p(x)
$$

具体的な例として，下図のように青色（右側）とオレンジ色（左側）の二つの正規分布を考えてみましょう．それぞれの正規分布の平均を $\mu_{\rm blue}, \mu_{\rm orange}$と表し，分散を$\sigma^2_{\rm blue}, \sigma^2_{\rm orange}$ と表すことにします．
あるデータ $x$ が青の正規分布から生成されたとすることの尤度を $\ell_{\rm blue}(x)$ とおき，オレンジの正規分布から生成されたとすることの尤度を $\ell_{\rm orange}(x)$ とおくと，それぞれ次式の通りとなります．
$$
\begin{aligned}
\ell_{\rm blue}(x) &= \frac{1}{\sqrt{2\pi\sigma_{\rm blue}^2}} \exp{\left( - \frac{(x-\mu_{\rm blue})^2}{2\sigma_{\rm blue}^2}\right)} \\
\ell_{\rm orange}(x) &= \frac{1}{\sqrt{2\pi\sigma_{\rm orange}^2}} \exp{\left( - \frac{(x-\mu_{\rm orange})^2}{2\sigma_{\rm orange}^2}\right)} \\
\end{aligned}
$$

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/likelihood.png">

このとき，図の $x_1$ については，$\ell_{\rm blue}(x_1) > \ell_{\rm orange}(x_1)$ が成り立ち（$x_1$は青の正規分布から生成されたとする方が尤もらしい），$x_2$ については，$\ell_{\rm blue}(x_2) < \ell_{\rm orange}(x_2)$ が成り立ちます（$x_2$はオレンジの正規分布から生成されたとする方が尤もらしい）．このように，尤度は，ある決まったパラメータを持つ分布がデータにうまく当てはまる度合いを表します．

データが $x_1, x_2, \ldots, x_N$ と複数ある場合の尤度は，$\ell(x_1, x_2, \ldots, x_N) = p(x_1, x_2, \ldots, x_N) $ と定義されます．独立性の仮定から，これは次式のように個別のデータに対する尤度の積の形になります．
$$
\begin{aligned}
\ell(x_1, x_2, \ldots, x_N) &= p(x_1)\cdot p(x_2)\cdot\cdots\cdot p(x_N) = \prod_{n=1}^{N} p(x_n)
\end{aligned}
$$


上の図のピンクの点と灰色の点がデータだった場合，$\ell_{\rm orange}(x_1, x_2, \ldots, x_n) >  \ell_{\rm blue}(x_1, x_2, \ldots, x_n) $ となり，これらのデータ全体はオレンジの方の正規分布から生成されたとする方がより尤もらしいと判断できます．

このような考え方に基づいて，尤度を規準としてデータから確率分布のパラメータを推定する方法のことを，**最尤推定**(maximum likelihood estimation)または**最尤法**(maximum likelihood method)といいます．
実は，式(1)と(2)は，この最尤推定によって得られます．

#### 1次元正規分布当てはめの最尤推定

以下，1次元正規分布当てはめの問題の最尤推定の結果が式(1),(2)となることを示します．与えられたデータ集合全体の尤度は上記のように個々のデータに対する尤度の積で書けます．これを最大にするようにパラメータ $\mu,\sigma^2$ を決めればよい，ということになるのですが，積で表した式は扱いにくいので，対数をとったものを考えます．
$$
\begin{aligned}
L &= \log{\ell(x_1, x_2, \ldots, x_N)} = \log\left( \prod_{n=1}^{N} p(x_n) \right) \\
&= \sum_{n=1}^N\log {p(x_n)}
\end{aligned}
$$
これを，**対数尤度**(log-likelihood)といいます．$\log$ は単調増加関数なので，尤度の最大化は対数尤度の最大化と等価です．

今は正規分布を考えていますので，対数尤度 $L$ は次のように変形できます．
$$
\begin{aligned}
L &= \sum_{n=1}^N \log\left( \frac{1}{\sqrt{2\pi\sigma^2}} \exp{\left( - \frac{(x_n-\mu)^2}{2\sigma^2}\right)} \right)\\
&= \sum_{n=1}^N \left( -\frac{1}{2}\log{\left(2\pi\sigma^2\right)} - \frac{(x_n-\mu)^2}{2\sigma^2}\right) \\
&= -\frac{N}{2}\log{2\pi} - \frac{N}{2}\log\sigma^2-\frac{1}{2\sigma^2}\sum_{n=1}^N(x_n-\mu)^2
\end{aligned}
$$


$L$を最大にする$\mu,\sigma^2$ を求めたい，ということなので，いつものように微分して $0$ とおく作戦を採ります．まず，
$$
\begin{aligned}
\frac{\partial L}{\partial \mu} &= -\frac{2}{2\sigma^2}\sum_{n=1}^N(x_n-\mu)\cdot(-1)\\
&= \frac{1}{\sigma^2}\sum_{n=1}^N(x_n-\mu) = \frac{1}{\sigma^2}\left( \sum_{n=1}^{N}x_n - N\mu\right) = 0
\end{aligned}
$$
より，式(1)が得られます．次に，
$$
\begin{aligned}
\frac{\partial L}{\partial \sigma^2} &= -\frac{N}{2}\frac{1}{\sigma^2}+\frac{1}{2(\sigma^2)^2}\sum_{n=1}^{N}(x_n-\mu)^2\\
&= \frac{1}{2\sigma^2}\left( \frac{1}{\sigma^2}\sum_{n=1}^{N}(x_n-\mu)^2 - N\right) = 0
\end{aligned}
$$
より，式(2)が得られます（注）．以上より，1次元正規分布当てはめ問題を最尤推定によって解くと，その解は式(1)と(2)となることが示されました．

※注: ここでは，$\sigma$ ではなく $\sigma^2$ をひとかたまりのものとして扱っています．

#### ［実験］1次元正規分布を当てはめてみよう

次のセルを実行すると，`N`で指定した数のデータに対して1次元正規分布を当てはめます．データは1000個の中からランダムに選びますので，同じ`N`でも実行の度に結果が変わります．

In [ ]:
#@title 1次元正規分布を当てはめよう
N = 10  #@param [10, 20, 50, 100] {type: "raw"}

# N 個のデータをランダムに抽出
X = dfHW['height'].sample(n=N).to_numpy()

# 平均と分散を推定
mu = np.mean(X)
sig2 = np.var(X)

# データのヒストグラムと推定された正規分布を描く
xx = np.linspace(140, 200, num=100)
px = norm.pdf(xx, loc=mu, scale=np.sqrt(sig2))
fig, ax = plt.subplots(1, facecolor="white", figsize=(8, 6))
ax.hist(X, density=True)
ax.plot(xx, px, linewidth=3, color='red')
ax.set_xlim(140, 200)
plt.show()
print(f'N = {N}  平均: {mu:.2f}  分散: {sig2:.2f}')


#### ★★ やってみよう ★★

上記のセルを何度も実行して結果を観察しよう

---
### 多変量正規分布の当てはめ

密度推定の問題は，データの次元数が2以上の場合にも同じように考えることができます．ここでは，$N$個の$D$次元ベクトル（$D\times 1$行列）から成るデータ集合
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
に$D$次元の**多変量正規分布**（注）
$$
p(\mathbf{x}) = \frac{1}{\sqrt{(2\pi)^D |\Sigma|}} \exp\left( -\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^{\top}\Sigma^{-1}(\mathbf{x} - \mathbf{\mu}) \right)$$
（$\mathbf{\mu}$ は$D$次元の平均ベクトル，$\Sigma$ は$D\times D$の分散共分散行列）を当てはめる問題を考えます．

※注: 多変量正規分布については，授業科目「多変量解析及び演習」ですでに学んだひとも多いと思いますので，詳しい説明は省略します．

1次元のときと同様に尤度を定めて最尤推定を考えると，尤度を最大にする最適なパラメータは次のように与えられます（証明は省略します）．
$$
\begin{aligned}
\mathbf{\mu} &= \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\\
\Sigma &= \frac{1}{N}\sum_{n=1}^{N}(\mathbf{x}_n-\mathbf{\mu})(\mathbf{x}_n-\mathbf{\mu})^{\top}
\end{aligned}
$$
それぞれ，データの標本平均と分散共分散行列そのものに一致しています．



#### ［よだんだよん］
多変量正規分布のパラメータは，$D$次元ベクトル$\mathbf{\mu}$ と $D\times D$行列$\Sigma$ です．$\mathbf{\mu}$ のパラメータ数は $D$ で，
$\Sigma$ のパラメータ数は $\frac{1}{2}D(D+1)$ です（$D^2$でないのは，$\Sigma$が対称行列だからです）．
データの次元数 $D$ が大きくなると，パラメータ数はその2乗のオーダーで大きくなります．
そのため，次元数の大きいデータでは，データ数が少ないとパラメータを安定して求めるのが難しかったりします．
詳しくは述べませんが，そのような場合には，以下のように $\Sigma$の形を制約することでパラメータ数を減らしたり，共分散行列の推定に正則化の手法を採り入れたり，主成分分析を利用して次元削減したりします．

$$
\begin{aligned}
\Sigma &= \textrm{diag}(\sigma_1^2, \sigma_2^2, \ldots, \sigma_D^2) = 
\begin{pmatrix}
\sigma_1^2 &  & 0\\
 & \ddots & \\
0 &  & \sigma_D^2
\end{pmatrix} \qquad \mbox{(対角行列に制約，パラメータ数$D$)}\\
\Sigma &= \sigma^2I =
\begin{pmatrix}
\sigma^2 &  & 0\\
 & \ddots & \\
0 &  & \sigma^2
\end{pmatrix} \qquad \mbox{(単位行列の定数倍に制約，パラメータ数$1$)}
\end{aligned}
$$

#### ［実験］多変量正規分布を当てはめてみよう

(身長, 体重)のデータに2次元の正規分布を当てはめる実験をやってみましょう．

In [ ]:
dfHW

`N`で指定した数のデータに対して2次元正規分布を当てはめます．データは1000個の中からランダムに選びますので，同じ`N`でも実行の度に結果が変わります．

In [ ]:
#@title 2次元正規分布を当てはめよう
N = 10  #@param [10, 20, 50, 100, 200, 500] {type: "raw"}

# N 個のデータをランダムに抽出
X = dfHW.sample(n=N).to_numpy()

# 平均と分散共分散行列を推定
mu = np.mean(X, axis=0)
cov = (X - mu).T @ (X - mu) / N

# グラフ描画用のグリッドデータの作成
xmin, xmax = 130, 210
ymin, ymax = 20, 100
xx, yy = np.mgrid[xmin:xmax:1, ymin:ymax:1]
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mu ,cov)

# データの散布図と推定された正規分布の等高線を描く
fig, ax = plt.subplots(facecolor='white', figsize=(8, 8))
ax.scatter(X[:, 0], X[:, 1])
ax.contour(xx, yy, zz, colors=['#ffa0a0', 'r'], levels=[0.0001, 0.001])
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()
print(f'N = {N}  平均: {mu}')
print(f'分散共分散行列:\n {cov}')

#### ★★ やってみよう ★★

上記のセルを何度も実行して結果を観察しよう